**Статистический машинный перевод** - вид машинного перевода, где эквивалент в языке-реципиенте подбирается на основании вероятности соответствия определенному токену в языке-источнике.

Формальное определение работы статистической модели основано на теореме Байеса:

P(Y|X) = (P(Y)*P(X|Y)) / P(X), где:

P(Y) - *априорная* вероятность (вероятность наступления события без учёта других событий)

P(X|Y) - *апостериорная* вероятность (вероятность наступления события с учётом того факта, что другое событие уже произошло)

В этом случае Y - единица на языке перевода, а X - единица на языке оригинала. 

Выделяют СМТ, основанный на:
1. Слове
2. Сегменте
3. Синтаксисе
4. Фразе (иерархический)

Принцип работы СМТ:
1. В начале собираются большие параллельные корпуса текстов, для которых мы хотим обучить модель, в качестве обучающей и тестовой выборок
2. Тексты предобрабатываются (токенизируются) и объединяются в последовательности в зависимости от того, на какой последовательности мы хотим обучать модель: если мы, например, имеем дело с биграммной моделью, токены объединяются в пары: i + i+1
3. Для последовательностей вычисляется вероятность совместной встречаемости в зависимости от того, насколько часто такие комбинации появляются в корпусах
4. Из набора токенов, имеющих ненулевую вероятность появления после токена i, выбирается тот, у которого вероятность наибольшая, благодаря чему осуществляется поиск наиболее вероятных комбинаций.
5. В зависимости от грамматики языков, на которых мы обучаем модель, выбираем наиболее корректный вариант перевода с точки зрения морфосинтаксической структуры, в том числе порядка слов. Этот процесс подбора называется *декодированием*.





Ниже приведён игрушечный пример обучения статистической модели на примере немецко-английского параллельного корпуса. Следует отметить, что качество обучения у модели нулевое, поскольку данных в корпусе слишком мало.

In [1]:
import tarfile

from sklearn.model_selection import train_test_split

from collections import Counter, defaultdict
import random

In [ ]:
with tarfile.open('de-en.tgz', 'r:gz') as tar:
  tar.extractall()

In [ ]:
with open('de-en.de', 'r', encoding='utf8') as f:
  german = f.read().split('\n')[:-1]

with open('de-en.en', 'r', encoding='utf8') as f:
  english = f.read().split('\n')[:-1]

print("Данные языка X:\n", german)
print("Данные языка Y:\n", english)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(english, german)

print("> Обучающая выборка:")
for text, label in zip(X_train, y_train):
    print(f"\nТекст на немецком: {label}\n Его перевод на английский: {text}\n")

print("> Тестовая выборка:")
for text, label in zip(X_test, y_test):
    print(f"\nТекст на немецком: {label}\n Его перевод на английский: {text}\n")

In [ ]:
def tokenize(sentences):
  # функция возвращает списки слов
  return [sentence.split() for sentence in sentences]

# токенизируем каждую выборку
X_train_tokens, X_test_tokens, y_train_tokens, y_test_tokens = tokenize(X_train), tokenize(X_test), tokenize(y_train), tokenize(y_test)

print('Образец токенизированного текста:', X_train_tokens)

In [ ]:
x_vocab = Counter(' '.join(german).split()).keys()
y_vocab = Counter(' '.join(english).split()).keys()

print(f"Словарь немецких словоформ: {x_vocab}\n Всего {len(x_vocab)} словоформ")
print(f"\nCловарь английских словоформ: {y_vocab}\n Всего {len(y_vocab)} словоформ")

In [ ]:
# вероятность того, что случайное слово x_vocab соответсвует случайному слову y_vocab
uniform = 1 / (len(x_vocab) * len(y_vocab))
result = format(uniform, '.10f') 
print(result)

In [ ]:
# t-model
t = {}

for i in range(len(X_train)):
  # начинаем итерацию по обучающей выборке
  for word_x in X_train_tokens[i]:
    for word_y in y_train_tokens[i]:
      # создаем t-table
      t[(word_x, word_y)] = uniform

# t-table
for elem in t:
  print("Соответствие |", elem[0], "  ->  ", elem[1], "| Вероятность:", format(t[elem], '.10f'))

In [21]:
# количество итераций обучения
epochs = 5

In [ ]:
for epoch in range(epochs):
  # начинаем обучение

  # шаг 0. создаем слоты для подсчета статистики
  count = {} # P(x|y)
  total = {} # P(y)

  for i in range(len(X_train)):
    # начинаем итерацию по обучающей выборке
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        # создаем слоты для подсчета условной вероятности совпадений в корпусе
        count[(word_x, word_y)] = 0
        # и слоты для статистической языковой модели y
        total[word_y] = 0

  # шаг 1. Expectation
  for i in range(len(X_train)):
    # начинаем итерацию по обучающей выборке
    total_stat = {} # статистика x

    # собираем предварительную статистику на основе данных x
    for word_x in X_train_tokens[i]:
      total_stat[word_x] = 0 # создаем слоты для подсчета статистики по каждому токену x
      for word_y in y_train_tokens[i]:
        # обновляем данные из t-table; увеличиваем значения при обнаружении совместной встречаемости
        total_stat[word_x] += t[(word_x, word_y)]

    # обновляем данные для P(x|y) и P(y)
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        # подсчет условной вероятности совпадений в корпусе: равномерное распределение / частотность x
        count[(word_x, word_y)] += t[(word_x, word_y)] / total_stat[word_x]
        # подсчет статистической информации y: равномерное распределение / частотность x
        total[word_y] += t[(word_x, word_y)] / total_stat[word_x]

  # шаг 2. Maximization
  for i in range(len(X_train)):
    # начинаем итерацию по обучающей выборке
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        # обновляем t-table: вероятность совпадения в корпусе / вероятность информации y
        t[(word_x, word_y)] = count[(word_x, word_y)] / total[word_y]

for elem in t:
  print("Соответствие |", elem[0], "  ->  ", elem[1], "| Вероятность:", round(t[elem], 3))

In [ ]:
# для обучения модели объединим 2 выборки
tokens = ' '.join(german).split()

# хранилище для биграмм
bigram_model = defaultdict(list)

# собираем все попарные совпадения
for i in range(len(tokens)-1):
    current_word = tokens[i]
    next_word = tokens[i + 1]
    bigram_model[current_word].append(next_word)

print(bigram_model)

def decoder(model, steps=5):
  # инициализация случайного токена
  current_word = random.choice(tokens)
  generated_sentence = current_word

  for step in range(steps):
    # пошаговая генерация
    print('Шаг', step+1)
    next_word_options = model[current_word]
    print(f'Правдоподобные варианты продолжения для токена {current_word}:', next_word_options)

    current_word = random.choice(next_word_options)
    generated_sentence += ' '
    generated_sentence += current_word
    print('Промежуточный результат:', generated_sentence)
    print()
  print('Результат:', generated_sentence)

decoder(bigram_model)

In [ ]:
# сортировка t-table по убыванию правдоподобия
sorted_t = sorted(t.items(), key = lambda k:(k[1], k[0]), reverse = True)

def translate(token):
  for element in sorted_t:
    if element[0][1] == token:
      # поиск совпадений в t-table
      return element[0][0]
    # если совпадений нет, функция будет возвращать None
    return None 

for sentence in y_test_tokens:
  print("Оригинальное предложение:", ' '.join(sentence))
  translation = list(map(lambda token: translate(token) or '', sentence)) # обрабатываем все случаи, включая те, когда совпадений в таблице найдено не было
  print("Перевод:", ' '.join(translation))

In [ ]:
from nltk.translate.bleu_score import corpus_bleu 
 
reference = [X_test_tokens[8]]  
 
candidate = [translate(token) for token in y_test_tokens[0]] 
 
bleu_score = corpus_bleu([reference], [candidate])  
 
print("BLEU Score:", format(bleu_score, '.10f'))

In [ ]:
reference

In [ ]:
candidate